## Problem Statement: 
### Take the data from all of the user's bank accounts (using Yolt app export of transactions csv file), modify the data to edit wrong categorisation and create a web page to show charts summarising personal expenses and net worth

In [ ]:
import pandas as pd                       #to perform data manipulation and analysis
import numpy as np                        #to cleanse data
from datetime import datetime             #to manipulate dates
import plotly.express as px               #to create interactive charts
import plotly.graph_objects as go         #to create interactive charts
from jupyter_dash import JupyterDash      #to build Dash apps from Jupyter environments
import dash_core_components as dcc        #to get components for interactive user interfaces
import dash_html_components as html       #to compose the dash layout using Python structures

#### Open old transaction file

In [ ]:
df = pd.read_csv("transactions.csv") 
df[6:10]

#### Data Preprocessing

In [ ]:
df.drop('account_id', axis=1, inplace=True)

df.drop('transaction_id', axis=1, inplace=True)

df['category'] = np.where(df['description'].str.contains('Lorenzo Rosa'),'Internal', df['category'] )

#Remove rows containing specific strings
df = df[df.category != "Internal"]   
df = df[df.description != "Exchanged to EUR"] 
df = df[df.description != "Exchanged to GBP"]    
df = df[df.category != "Investments"] 
df = df[df.description != "To GBP Savings"] 
df = df[df.description != "From GBP Savings"]     #check'Exchanged from and to GBP'

#Personalise your categories
df['category'] = np.where(df['description'].str.contains('Amazon|AMAZON|Amz'),'Materialistic Desire', df['category'])

df['category'] = np.where(df['description'].str.contains('Gym|GYM|Thenx|Footy|Sporting'),'Sport', df['category'] )

df['category'] = np.where(df['description'].str.contains('George MacLean'),'Rent&Bills', df['category'] )

df['category'] = np.where(df['description'].str.contains('UBER|Uber|uber|ZIPCAR|Zipcar|bird|Lim|TFL TRAVEL|Tfl Travel Charge|Ewa'), 'Transport', df['category'] )

df['category'] = np.where(df['description'].str.contains('Wooden|Hackney Cycles'),'Materialistic Desire', df['category'] )

df['category'] = np.where(df['description'].str.contains('itunes|apple|ODEON|Odeon'),'Entertainment', df['category'] )

df['category'] = np.where(df['description'].str.contains('Airbnb|Ryanair|Trainline|trainline|Booking|Flixbus'),'Travel', df['category'] )

df['category'] = np.where(df['merchant'] == 'Cash Withdrawal', 'Cash Withdrawal', df['category'])

df.shape


df['category'] = np.where(df['description'].str.contains('Uber|Zipcar|Bird'),'Transport',df['category'])


In [ ]:
#value = 'To '
df['starts'] = list( 
    map(lambda x: x.startswith('To '), df['description'])) 

df.loc[df.starts == True, 'category'] = "Transfer"
df.drop('starts', axis=1, inplace=True)


In [ ]:
df.category.replace(["Bills", "Expenses", "General", "Housing", "Leisure"], "Other",inplace=True)
df.category.replace(["Coffee", "Eating out", "Takeaway", "Lunch"], "Food",inplace=True)
df.category.replace("Education", "Personal Improvement",inplace=True)
df.category.replace("Drinks", "Entertainment",inplace=True)
df.category.replace("Petrol", "Transport",inplace=True)
df.category.replace("Shopping", "Materialistic Desire",inplace=True)
df.category.replace("Transfers", "Transfer",inplace=True)
df.category.replace("Cash", "Cash Withdrawal",inplace=True)
df.category.replace("Personal Improvement", "Self Improvement",inplace=True)


In [ ]:
#to remove
df.head(5)

In [ ]:
df['date'] = pd.to_datetime(df['date'])
df['year_month'] = df['date'].dt.strftime('%Y-%m')
df
df.head(5)

#### Export cleansed csv file

In [ ]:
#filename = datetime.now().strftime("%Y-%m-%d-%H-%M.csv")

#path = "C:/Users/lorenzorosa/Desktop/Python Files/Personal Budget/NewTransactionFile " +   filename 
#df.to_csv (path)


#### Net Worth Over Time Chart

In [ ]:
Net_Worth_Table = df.groupby('year_month')['amount'].sum().reset_index(name ='sum')
Net_Worth_Table['cumulative sum'] = Net_Worth_Table['sum'].cumsum()
Net_Worth_Table

In [ ]:
Net_Worth_Chart = go.Figure(
    data = go.Scatter(x = Net_Worth_Table["year_month"], y = Net_Worth_Table["cumulative sum"]),
    layout = go.Layout(
        title = go.layout.Title(text="Net Worth Over Time")
    )
)
Net_Worth_Chart.update_layout(
    xaxis_title = "Date",
    yaxis_title = "Net Worth (£)",
    hovermode = 'x unified'
    )
Net_Worth_Chart.update_xaxes(
    tickangle = 45)
Net_Worth_Chart.show()

#### Total Monthly Expenses Chart

In [ ]:
df = df[df.category != "Income"] 
df.amount = df.amount*(-1) 

In [ ]:
Total_Monthly_Expenses_Table = df.groupby('year_month')['amount'].sum().reset_index(name='sum')
Total_Monthly_Expenses_Table = Total_Monthly_Expenses_Table.rename(columns={'year_month': 'DATE', 'sum': 'TOTAL EXPENSE'})
Total_Monthly_Expenses_Table

In [ ]:
Total_Monthly_Expenses_Chart = px.bar(Total_Monthly_Expenses_Table, x="DATE", y="TOTAL EXPENSE",title="Total Monthly Expenses")
Total_Monthly_Expenses_Chart.update_yaxes(title='Expenses (£)', visible=True, showticklabels=True)
Total_Monthly_Expenses_Chart.update_xaxes(title='Date',visible=True, showticklabels=True)

Total_Monthly_Expenses_Chart.show()

#### Expenses Breakdown Chart

In [ ]:
Expenses_Breakdown_Table = pd.pivot_table(df, values = ['amount'], index = ['category', 'year_month'], aggfunc=sum).reset_index()
Expenses_Breakdown_Table.columns = [x.upper() for x in Expenses_Breakdown_Table.columns]
Expenses_Breakdown_Table = Expenses_Breakdown_Table.rename(columns={'YEAR_MONTH': 'DATE'})
Expenses_Breakdown_Table = Expenses_Breakdown_Table[['DATE', 'CATEGORY', 'AMOUNT']]


In [ ]:
#Creation of a df with all dates within the range we have data for each category /
#(e.g. for cash withdrawal there are no transactions between MArch 2020 and August 2020 /
#so there is not data point in the graph for those dates and I have to fix that)

Expenses_Breakdown_Table_All_Dates = Expenses_Breakdown_Table.set_index(
    ['DATE', 'CATEGORY']
).unstack(
    fill_value=0
).asfreq(
    'M', fill_value=0
).stack().sort_index(level=1).reset_index()

Expenses_Breakdown_Table_All_Dates['DATE'] = pd.to_datetime(Expenses_Breakdown_Table_All_Dates['DATE'])
Expenses_Breakdown_Table_All_Dates['DATE'] = Expenses_Breakdown_Table_All_Dates['DATE'].dt.strftime('%Y-%m')

In [ ]:
#Appending all dates to the original df

Expenses_Breakdown_Table_Final = Expenses_Breakdown_Table.append(Expenses_Breakdown_Table_All_Dates, ignore_index=True)
Expenses_Breakdown_Table_Final = Expenses_Breakdown_Table_Final.drop_duplicates(subset = ['DATE', 'CATEGORY'], keep = 'first')
Expenses_Breakdown_Table_Final = Expenses_Breakdown_Table_Final.sort_values(['DATE', 'CATEGORY'], ascending=[True, False],ignore_index=True)

In [ ]:
#creating a df only for the latest date
#I need it to add 0s in case in the latest date there was no transaction for a specific category
#otherwise again the data point for the latest date will not appear in the chart
#(before I fixed only the dates within the date range of the transactions, not the latest date)

latest_date = Expenses_Breakdown_Table_Final['DATE'].max()
df_latest_date = Expenses_Breakdown_Table_Final.loc[Expenses_Breakdown_Table_Final['DATE'] == latest_date]

In [ ]:
categories_lst = ['Travel','Transport','Transfer','Other','Materialistic Desire','Groceries','Gifts','Food','Entertainment','Cash Withdrawal','Rent&Bills','Personal Care','Sport']
missing_cat_latest_date = pd.DataFrame({'CATEGORY': list(set(df_latest_date['CATEGORY']) ^ set(categories_lst))})
missing_cat_latest_date['AMOUNT']=0.0
missing_cat_latest_date['DATE']= df_latest_date['DATE'].max()
missing_cat_latest_date = missing_cat_latest_date [['DATE', 'CATEGORY', 'AMOUNT']]
missing_cat_latest_date

In [ ]:
#Appending the categories with no transactions for the latest date to the final df for this chart

Expenses_Breakdown_Table_Final = Expenses_Breakdown_Table_Final.append (missing_cat_latest_date)

In [ ]:
Expenses_Breakdown_Chart = px.line(Expenses_Breakdown_Table_Final, x='DATE', y="AMOUNT", title="Expenses Breakdown", color = 'CATEGORY')
Expenses_Breakdown_Chart.update_yaxes(title='Expenses (£)', visible=True, showticklabels=True)
Expenses_Breakdown_Chart.update_xaxes(title='Date', visible=True, showticklabels=True)

Expenses_Breakdown_Chart.show()

In [ ]:
# Build App
app = JupyterDash(__name__)

app.layout = html.Div([
    
    html.Div([
        html.H1(str(latest_date)+" Personal Finance Summary",style={'text-align':'center'}),
        dcc.Graph(figure = Net_Worth_Chart)
    ]),
  
    html.Div([
        dcc.Graph(figure = Total_Monthly_Expenses_Chart)
    ]),
    
    html.Div([
        dcc.Graph(figure = Expenses_Breakdown_Chart)

    ])
])
    
# Run app and display result
app.run_server(mode='external')

#CLick on the link below to access the "Personal Finances Summary"

TESTS

In [ ]:
#follow to design charts
#https://pbpython.com/plotly-dash-intro.html

In [ ]:
#df.loc[df['description'] == "Uber Trip Help.uber.com"]

In [ ]:
#OLD WORKING CODE FOR THE EXPENSES BREAKDOWN CHART
# Expenses_Breakdown_Table = pd.pivot_table(df, values = ['amount'], index = ['category', 'year_month'], aggfunc=sum).reset_index()
# Expenses_Breakdown_Table.columns = [x.upper() for x in Expenses_Breakdown_Table.columns]
# Expenses_Breakdown_Table = Expenses_Breakdown_Table.rename(columns={'YEAR_MONTH': 'DATE'})

# Expenses_Breakdown_Table

In [ ]:
# Expenses_Breakdown_Chart = px.line(Expenses_Breakdown_Table, x='DATE', y="AMOUNT", title="Expenses Breakdown", color = 'CATEGORY')
# Expenses_Breakdown_Chart.update_yaxes(title='Expenses (£)', visible=True, showticklabels=True)
# Expenses_Breakdown_Chart.update_xaxes(title='Date', visible=True, showticklabels=True)

# Expenses_Breakdown_Chart.show()

In [ ]:
Expenses_Breakdown_Table_TEST1 = Expenses_Breakdown_Table.copy()
Expenses_Breakdown_Table_TEST1 = Expenses_Breakdown_Table_TEST1[['DATE', 'CATEGORY', 'AMOUNT']]
Expenses_Breakdown_Table_TEST1.head(20)

In [ ]:
Expenses_Breakdown_Table_TEST2 = Expenses_Breakdown_Table_TEST1.set_index(
    ['DATE', 'CATEGORY']
).unstack(
    fill_value=0
).asfreq(
    'M', fill_value=0
).stack().sort_index(level=1).reset_index()


In [ ]:
Expenses_Breakdown_Table_TEST2['DATE'] = pd.to_datetime(Expenses_Breakdown_Table_TEST2['DATE'])
Expenses_Breakdown_Table_TEST2['DATE'] = Expenses_Breakdown_Table_TEST2['DATE'].dt.strftime('%Y-%m')

In [ ]:
Expenses_Breakdown_Table_TEST1.head(15)

In [ ]:
Expenses_Breakdown_Table_TEST2.head(15)

In [ ]:
test3 = Expenses_Breakdown_Table_TEST1.append(Expenses_Breakdown_Table_TEST2, ignore_index=True)
test3 = test3.drop_duplicates(subset = ['DATE', 'CATEGORY'], keep = 'first')
test3 = test3.sort_values(['DATE', 'CATEGORY'], ascending=[True, False],ignore_index=True)

test3.loc[test3['DATE'] == '2020-09']


In [ ]:
recent_date = test3['DATE'].max()
df3 = test3.loc[test3['DATE'] == recent_date]
df3

In [ ]:
recent_date

In [ ]:
lst = ['Travel','Transport','Transfer','Other','Materialistic Desire','Groceries','Gifts','Food','Entertainment','Cash Withdrawal','Rent&Bills','Personal Care','Sport']
missing = pd.DataFrame({'CATEGORY': list(set(df3['CATEGORY']) ^ set(lst))})
missing['AMOUNT']=0.0
missing['DATE']= df3['DATE'].max()
missing = missing [['DATE', 'CATEGORY', 'AMOUNT']]
missing

In [ ]:
test3 = test3.append (missing)

In [ ]:
Test_Chart = px.line(test3, x='DATE', y="AMOUNT", title="Expenses Breakdown", color = 'CATEGORY')
Test_Chart.update_yaxes(title='Expenses (£)', visible=True, showticklabels=True)
Test_Chart.update_xaxes(title='Date', visible=True, showticklabels=True)

Test_Chart.show()

In [ ]:
Expenses_Breakdown_Chart.show()